In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_hist_trans = pd.read_csv('data/hist_txn_tmp.csv')
df_new_merchant_trans = pd.read_csv('data/new_merchant_txn_tmp.csv')

In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

    '''
    for k in aggs.keys():
        for agg in aggs[k]:
            name+_+k+agg
    '''

In [4]:
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

In [5]:
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

In [6]:
aggs

{'authorized_flag': ['sum', 'mean'],
 'card_id': ['size'],
 'category_1': ['sum', 'mean'],
 'dayofweek': ['nunique'],
 'hour': ['nunique'],
 'installments': ['sum', 'max', 'min', 'mean', 'var'],
 'merchant_category_id': ['nunique'],
 'merchant_id': ['nunique'],
 'month': ['nunique'],
 'month_diff': ['mean'],
 'month_lag': ['max', 'min', 'mean', 'var'],
 'purchase_amount': ['sum', 'max', 'min', 'mean', 'var'],
 'purchase_date': ['max', 'min'],
 'subsector_id': ['nunique'],
 'weekend': ['sum', 'mean'],
 'weekofyear': ['nunique'],
 'year': ['nunique']}

In [7]:
for col in ['category_2','category_3']:
    df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']  

In [8]:
df_hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,subsector_id,year,month,dayofweek,weekofyear,hour,weekend,month_diff,category_2_mean,category_3_mean
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,...,37,2017,6,6,25,15,1,11,0.074568,0.358458
1,1,C_ID_4e6213e9bc,88,0,0,A,367,M_ID_86ec983688,-7,-0.733128,...,16,2017,7,5,28,12,1,12,0.074568,0.358458
2,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_979ed661fc,-6,-0.720386,...,37,2017,8,2,32,22,0,12,0.074568,0.358458
3,1,C_ID_4e6213e9bc,88,0,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,...,34,2017,9,5,35,10,1,12,0.074568,0.358458
4,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-11,-0.722865,...,37,2017,3,4,10,1,0,12,0.074568,0.358458


In [10]:
df_hist_trans.category_2.unique()

array([1., 3., 5., 2., 4.])

In [11]:
df_hist_trans.category_3.unique()

array(['A', 'B', 'C'], dtype=object)

In [12]:
df_hist_trans.category_1.unique()

array([0, 1])

In [14]:
df_hist_trans.groupby('category_1')['purchase_amount'].mean()

category_1
0    0.071540
1   -0.419327
Name: purchase_amount, dtype: float64

In [15]:
new_columns = get_new_columns('hist',aggs)

In [16]:
new_columns

['hist_month_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_dayofweek_nunique',
 'hist_year_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_lag_var',
 'hist_month_diff_mean',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_weekend_sum',
 'hist_weekend_mean',
 'hist_category_1_sum',
 'hist_category_1_mean',
 'hist_card_id_size',
 'hist_category_2_mean_mean',
 'hist_category_3_mean_mean']

In [18]:
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)

In [21]:
#card_id': ['size'],
df_hist_trans.groupby('card_id').agg('size')

card_id
C_ID_00007093c1    149
C_ID_0001238066    123
C_ID_0001506ef0     66
C_ID_0001793786    216
C_ID_000183fdda    144
                  ... 
C_ID_ffff1d9928     12
C_ID_ffff579d3a    114
C_ID_ffff756266     24
C_ID_ffff828181    190
C_ID_fffffd5772     84
Length: 325540, dtype: int64

In [22]:
df_hist_trans[df_hist_trans['card_id']=='C_ID_00007093c1']

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,subsector_id,year,month,dayofweek,weekofyear,hour,weekend,month_diff,category_2_mean,category_3_mean
19095774,1,C_ID_00007093c1,244,0,1,B,307,M_ID_9400cf2342,-4,-0.530526,...,19,2017,10,0,40,19,0,12,0.180375,-0.404556
19095775,1,C_ID_00007093c1,244,0,1,B,705,M_ID_a33355a1b7,-12,-0.467415,...,33,2017,2,1,7,15,0,12,0.180375,-0.404556
19095776,0,C_ID_00007093c1,244,0,1,B,108,M_ID_15446e939c,-9,0.307953,...,34,2017,5,3,20,16,0,12,0.180375,-0.404556
19095777,1,C_ID_00007093c1,76,0,3,C,45,M_ID_84e60960a3,-6,-0.559076,...,18,2017,8,5,31,11,1,12,0.180375,0.106023
19095778,1,C_ID_00007093c1,-1,1,1,B,755,M_ID_57df19bf28,-7,-0.709342,...,8,2017,7,0,27,15,0,12,0.074568,-0.404556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095918,1,C_ID_00007093c1,244,0,1,B,307,M_ID_9400cf2342,-2,-0.701753,...,19,2017,12,2,50,20,0,12,0.180375,-0.404556
19095919,1,C_ID_00007093c1,244,0,1,B,307,M_ID_5357547c3e,-4,-0.515500,...,19,2017,10,4,40,21,0,12,0.180375,-0.404556
19095920,0,C_ID_00007093c1,244,0,1,B,307,M_ID_9400cf2342,-1,-0.521510,...,19,2018,1,0,2,18,0,12,0.180375,-0.404556
19095921,1,C_ID_00007093c1,244,0,1,B,705,M_ID_8261f7a734,-4,-0.586124,...,33,2017,10,0,40,17,0,12,0.180375,-0.404556


In [19]:
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)

- calculate the difference between min and max purchase date
- calculate the ratio of purchase duration over number of transactions
- calculate the duration of now and last purchase

In [24]:
df_hist_trans_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Data columns (total 35 columns):
card_id                              325540 non-null object
hist_month_nunique                   325540 non-null int64
hist_hour_nunique                    325540 non-null int64
hist_weekofyear_nunique              325540 non-null int64
hist_dayofweek_nunique               325540 non-null int64
hist_year_nunique                    325540 non-null int64
hist_subsector_id_nunique            325540 non-null int64
hist_merchant_id_nunique             325540 non-null int64
hist_merchant_category_id_nunique    325540 non-null int64
hist_purchase_amount_sum             325540 non-null float64
hist_purchase_amount_max             325540 non-null float64
hist_purchase_amount_min             325540 non-null float64
hist_purchase_amount_mean            325540 non-null float64
hist_purchase_amount_var             325540 non-null float64
hist_installments_sum                325540 non-null

In [29]:
(pd.to_datetime(df_hist_trans_group['hist_purchase_date_max'])- pd.to_datetime(df_hist_trans_group['hist_purchase_date_min'])).dt.days

0         377
1         151
2         398
3         283
4         202
         ... 
325535     89
325536    199
325537    260
325538    410
325539    188
Length: 325540, dtype: int64

In [31]:
df_hist_trans_group['hist_purchase_date_diff'] = (pd.to_datetime(df_hist_trans_group['hist_purchase_date_max'])- pd.to_datetime(df_hist_trans_group['hist_purchase_date_min'])).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - pd.to_datetime(df_hist_trans_group['hist_purchase_date_max'])).dt.days

In [32]:
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

252

Do the same for new transactions

In [33]:
for col in ['category_2','category_3']:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean']
    
new_columns = get_new_columns('new_hist',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)

In [34]:
df_hist_trans_group['new_hist_purchase_date_diff'] = (pd.to_datetime(df_hist_trans_group['new_hist_purchase_date_max']) - pd.to_datetime(df_hist_trans_group['new_hist_purchase_date_min'])).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - pd.to_datetime(df_hist_trans_group['new_hist_purchase_date_max'])).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

27

In [35]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,new_hist_weekend_sum,new_hist_weekend_mean,new_hist_category_1_sum,new_hist_category_1_mean,new_hist_card_id_size,new_hist_category_2_mean_mean,new_hist_category_3_mean_mean,new_hist_purchase_date_diff,new_hist_purchase_date_average,new_hist_purchase_date_uptonow
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,9,23,35,7,...,6.0,0.260870,0.0,0.000000,23.0,-0.550160,-0.592993,54.0,2.347826,938.0
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,12,24,50,7,...,0.0,0.000000,0.0,0.000000,6.0,-0.550160,-0.606486,56.0,9.333333,968.0
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,10,14,22,7,...,1.0,1.000000,0.0,0.000000,1.0,-0.549015,-0.592993,0.0,0.000000,938.0
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,6,16,20,7,...,3.0,0.428571,1.0,0.142857,7.0,-0.556518,-0.604559,41.0,5.857143,949.0
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,22,17,7,...,12.0,0.333333,2.0,0.055556,36.0,-0.555446,-0.588217,57.0,1.583333,938.0


In [39]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_purchase_date_max'] = pd.to_datetime(df['hist_purchase_date_max'])
    df['hist_purchase_date_min'] = pd.to_datetime(df['hist_purchase_date_min'])
    df['new_hist_purchase_date_max'] = pd.to_datetime(df['new_hist_purchase_date_max'])
    df['new_hist_purchase_date_min'] = pd.to_datetime(df['new_hist_purchase_date_min'])
    
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9 #a way to change it from datetime to a numeric value that the model could use
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

In [42]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()


0    199710
1      2207
Name: outliers, dtype: int64

replace feature 1 with the mean of each value of feature 1: the smaller the mean, the less outliers it has

In [50]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)

In [52]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']

In [57]:
df_train['target']

KeyError: 'target'

In [53]:
df_train_columns

['feature_1',
 'feature_2',
 'feature_3',
 'hist_month_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_dayofweek_nunique',
 'hist_year_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_lag_var',
 'hist_month_diff_mean',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_weekend_sum',
 'hist_weekend_mean',
 'hist_category_1_sum',
 'hist_category_1_mean',
 'hist_card_id_size',
 'hist_category_2_mean_mean',
 'hist_category_3_mean_mean',
 'hist_purchase_date_diff',
 'hist_purchase_date_average'

In [54]:
del df_train['target']

In [58]:
df_train2 = pd.read_csv('data/train.csv')

In [59]:
df_train['target'] = df_train2['target']

In [60]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_year_nunique,...,dayofweek,weekofyear,month,elapsed_time,hist_first_buy,new_hist_first_buy,card_id_total,purchase_amount_total,outliers,target
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,9,23,35,7,2,...,3,22,6,1270,26,277.0,283.0,-179.212942,0,-0.820283
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,12,24,50,7,2,...,6,52,1,1421,5,396.0,356.0,-214.362071,0,0.392913
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,10,14,22,7,2,...,0,31,8,1574,163,635.0,44.0,-29.867717,0,0.688056
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,6,16,20,7,2,...,4,35,9,1178,25,187.0,84.0,-54.145736,0,0.142495
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,4,22,17,7,2,...,2,44,11,1117,11,121.0,169.0,-68.613893,0,-0.159749


In [61]:
df_train.to_csv('data/processed_train_'+ pd.datetime.now().strftime("%d%m%Y_%H%M") + '.csv',index=False)
df_test.to_csv('data/processed_test_'+ pd.datetime.now().strftime("%d%m%Y_%H%M") + '.csv',index=False)

In [63]:
df_train.shape

(201917, 89)

In [65]:
df_train.outliers.unique()

array([0, 1])